In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import logging
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# Import from scripts
sys.path.append("..") 
from src.data_loader import load_tsla_data
#from scripts.arima import train_arima, forecast_arima, evaluate_arima, plot_results,preprocess_data

In [3]:
tesla_data = load_tsla_data('../data/TSLA_cleaned.csv')  



2025-03-01 22:38:08,965 - INFO - Loading TSLA data from CSV...
2025-03-01 22:38:09,083 - INFO - Verifying TSLA data integrity...
2025-03-01 22:38:09,086 - INFO - TSLA data loaded successfully. Shape: (2505, 9), Missing values: 0


In [4]:
#tesla_data = preprocess_data(tesla_data)
tesla_data.head() 

,Price,Close,High,Low,Volume,Daily_Return,Rolling_Mean,Rolling_Std,Z_Score
Date,,,,,,,,,
2015-02-17,13.623333,13.713333,13.433333,13.713333,59694000,-0.140787,13.940933,0.583636,-0.895868
2015-02-18,13.630667,13.744667,13.506667,13.611333,40704000,0.228492,13.917978,0.577107,-0.895605
2015-02-19,14.114000,14.162667,13.583333,13.666667,77311500,3.041181,13.914067,0.574963,-0.892099
2015-02-20,14.474000,14.506667,13.987333,14.052000,89731500,2.428920,13.920333,0.580523,-0.889213
2015-02-23,13.822667,14.546667,13.755333,14.377333,127497000,0.275735,13.930111,0.588737,-0.888878
